In [1]:
import os, sys
os.chdir('../')

In [2]:
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import loss as module_loss
import model.metric as module_metric
import model.model as module_arch

import easydict
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

import data_loader.data_loaders as module_data
import model.model as module_arch

from selection.svd_classifier import *
from selection.gmm import *
from selection.util import *

from utils.parse_config import ConfigParser
from utils.util import *
from utils.args import *

In [3]:
config_file = './hyperparams/multistep/config_cifar10_cce_rn34.json'
with open(config_file, 'r') as f:
    config = json.load(f)

# resume_path = './rn34/multistep_asym_40_elr.pth'

In [4]:
def decode(path):
    items = path.split('_')
    noisetype = True if items[1]=='asym' else False
    noiserate = float(items[2]) * 0.01
    
    return noisetype, noiserate, items[3].split('.')[0]

In [5]:
def make_parse(resume_path, config, noise_rate, noisetype):
    parse = easydict.EasyDict({
    "load_name" : resume_path,
    "reinit": False,
    "distill_mode": 'fine-kmeans'
    })
    
    config['trainer']['percent'] = noise_rate
    config['trainer']['asym'] = noisetype
    
    return parse, config

In [10]:
def return_statistics(dataloader, clean_labels, datanum):
    predict = np.zeros(datanum)
    for idx in clean_labels: predict[idx] = 1
        
    isNoisy_list = compute_noiseratio(dataloader)
    r_stats = []
    
    tp = (isNoisy_list[predict==1]==1).sum() # positive clean
    tn = (isNoisy_list[predict==0]==0).sum() # negative noisy
    fp = (isNoisy_list==0).sum() - tn
    fn = (isNoisy_list==1).sum() - tp
    
    print('Noisy: {}, Clean: {}'.format((isNoisy_list==0).sum(), (isNoisy_list==1).sum()))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sel_samples = int(fp + tp)
    frac_clean = tp / (fp + tp)
    F1 =  tp / (tp + (fp+fn)/2)

    r_stats.extend([sel_samples, round(precision, 4), round(recall, 4), round(specificity, 4), round(accuracy, 4), round(frac_clean, 4)])
    print('Selected samples: {} \nPrecision: {} \nRecall: {} \nSpecificity: {}\nAccuracy: {} \nFraction of clean samples/selected samples: {}'.format(r_stats[0], r_stats[1], r_stats[2], r_stats[3], r_stats[4], r_stats[5]))
    
    return r_stats[0], r_stats[1], r_stats[2], r_stats[3], r_stats[4], F1

In [7]:
def extract_cleanidx(teacher, data_loader, parse, print_statistics = True):
    teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    teacher = teacher.cuda()

    if not parse.reinit: teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    for params in teacher.parameters(): params.requires_grad = False
    
    if 'fine' in parse.distill_mode:
        features, labels = get_features(teacher, data_loader)
        clean_labels = fine(current_features=features, current_labels=labels, fit = parse.distill_mode)
    elif 'loss' in parse.distill_mode:
        clean_labels, labels = cleansing_loss(teacher, data_loader)
    else:
        raise NotImplemented 
    if print_statistics: 
        selected, precision, recall, specificity, accuracy, F1 = return_statistics(data_loader, clean_labels, datanum=len(labels))
    
    return selected, precision, recall, specificity, accuracy, F1

In [8]:
def make_pd_list(root, config, log_filename):
    random.seed(config['seed'])
    torch.manual_seed(config['seed'])
    torch.cuda.manual_seed_all(config['seed'])
    torch.backends.cudnn.deterministic = True
    np.random.seed(config['seed'])
    
    # load checkpoint path
    pathlist = os.listdir(root)
    pathlist = [path for path in pathlist if ('.pth' in path) and ('eigen' not in path) and ('kmeans' not in path) and ('c100') not in path]
    
    # initialize model
    model = module_arch.resnet34(num_classes=10)
    
    # make pandas file
    logcolumns = ['noisetype', 'noiserate', 'lossfunction', 'selected', 'precision', 'recall', 'specificity', 'accuracy', 'F1']
    log_pd = pd.DataFrame(np.zeros([len(pathlist), len(logcolumns)]), columns = logcolumns)
    
    # write pandas file
    noisetypelst = ['']
    for i in range(len(pathlist)):
        noisetype, noiserate, lossfunction = decode(pathlist[i])
        parse, config = make_parse('./rn34/' + pathlist[i], config, noiserate, noisetype)
        
        # load original dataloader
        data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size= 100,
        shuffle=False,
        validation_split=0.0,
        num_batches=config['data_loader']['args']['num_batches'],
        training=True,
        num_workers=config['data_loader']['args']['num_workers'],
        pin_memory=config['data_loader']['args']['pin_memory'],
        config=config)
        
        selected, precision, recall, specificity, accuracy, F1 = extract_cleanidx(model, data_loader, parse)
        log_pd.loc[i] = [str(noisetype), str(noiserate), lossfunction, selected, precision, recall, specificity, accuracy, F1]
        log_pd.to_csv(log_filename)
        
    return log_pd

In [ ]:
make_pd_list(root = './checkpoint/rn34/', config=config, log_filename = 'c10_n_fine_pretrained_statistics.csv')

Files already downloaded and verified
##############
[3 2 1 1 3 0 0 7 6 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


100%|██████████| 500/500 [00:04<00:00, 105.88it/s]


Noisy: 9226, Clean: 40774
Selected samples: 40837 
Precision: 0.9349 
Recall: 0.9364 
Specificity: 0.7119
Accuracy: 0.8949 
Fraction of clean samples/selected samples: 0.9349
Files already downloaded and verified
##############
[8 9 1 9 4 8 3 6 3 6]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 125.98it/s]


Noisy: 36036, Clean: 13964
Selected samples: 12962 
Precision: 0.8709 
Recall: 0.8084 
Specificity: 0.9536
Accuracy: 0.913 
Fraction of clean samples/selected samples: 0.8709
Files already downloaded and verified
##############
[3 2 1 1 3 2 2 7 6 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


...

100%|██████████| 500/500 [00:04<00:00, 110.79it/s]


Noisy: 2458, Clean: 47542
Selected samples: 45035 
Precision: 0.9969 
Recall: 0.9443 
Specificity: 0.943
Accuracy: 0.9443 
Fraction of clean samples/selected samples: 0.9969
Files already downloaded and verified
##############
[3 9 1 1 3 8 2 7 6 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


100%|██████████| 500/500 [00:04<00:00, 119.43it/s]


Noisy: 9006, Clean: 40994
Selected samples: 39960 
Precision: 0.9979 
Recall: 0.9728 
Specificity: 0.9908
Accuracy: 0.976 
Fraction of clean samples/selected samples: 0.9979
Files already downloaded and verified
##############
[8 9 1 9 3 8 2 7 3 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


 40%|████      | 202/500 [00:12<00:26, 11.04it/s]